# Three-step RAS expansion for the $1s^2 2s^2\: ^1S_0$ and $1s^2 2s2p\: ^{1,3}P_1$ levels of neutral beryllium

In [28]:
using JAC


In JAC, an restricted-active space (RAS) expansion is a particular (atomic) representation and, more generally, refers to a systematically enlarged SCF computation. Such a computation is performed stepwise in order to include additional layers of active orbitals. RAS computations are based on one or several *reference configurations*, a well-defined *level symmetry* of all considered CSF bases as well as on virtual single- (S), double- (D), triple-excitations (T), etc. of electrons from valence orbitals $\to$ valence+virtual orbitals from the (so-called) *active* set. These SD++ excitations can be chosen differently for each step of the RAS computation, similar as the orbitals which are kept *frozen* during a particular step. More often than not, another layer of active orbitals is added and optimized in each step, while all orbitals from previous steps are kept frozen.

In this tutorial, we explain how a three-step RAS expansion for the $1s^2 2s^2\: ^1S_0$ ground level of neutral beryllium as well as for its two lowest-excited $1s^2 2s2p\: ^{1,3}P_1$ levels can be readily generated. For such a representation (of one or several atomic states), we first compute the orbitals for the $1s^2 2s^2$ reference configuration (in some configuration-average approach), and then stepwise enlarge the active orbitals by $2s,\: 2p$ (step$_1$), $3s,\: 3p,\: 3d$ (step$_2$), and finally $4s,\: ...,\: 4f$ (step$_3$). In all these steps, the $1s$ orbitals as well as the orbitals from the previous steps are kept frozen to ensure a proper convergence of the orbitals.

Before we shall explain further details about such RAS expansions (and computations), however, let us however look again at the `struct AtomicState.Representation` from which the RAS representation can be *generate*(d) as usual:

In [2]:
? AtomicState.Representation

`struct  AtomicState.Representation`       ... a struct for defining an atomic state representation. Such representations often refer to approximate wave function approximations of         one or several levels but may concern also a mean-field basis (for some multiplet of some given configurations) or Green functions,         etc.

```
+ name             ::String                      ... to assign a name to the given model.
+ nuclearModel     ::Nuclear.Model               ... Model, charge and parameters of the nucleus.
+ grid             ::Radial.Grid                 ... The radial grid to be used for the computation.
+ refConfigs       ::Array{Configuration,1}      ... List of references configurations, at least 1.
+ repType          ::AbstractRepresentationType  ... Specifies the particular representation.
```

---

`AtomicState.Representation()`  ... constructor for an 'empty' instance of the a variable::AtomicState.Representation

---

`AtomicState.Representation( ... example for the generation of a mean-field basis)`  

```
    name        = "Oxygen 1s^2 2s^2 2p^4 ground configuration"
    grid        = Radial.Grid(true)
    nuclearM    = Nuclear.Model(8.)
    refConfigs  = [Configuration("[He] 2s^2 2p^4")]
    mfSettings  = MeanFieldSettings()
    Representation(name, nuclearM, grid, refConfigs, MeanFieldBasis(mfSettings) )
```

`AtomicState.Representation( ... example for the computation of a configuration-interaction (CI) expansion)`  

```
    name        = "Oxygen 1s^2 2s^2 2p^4 ground configuration"
    grid        = Radial.Grid(true)
    nuclearM    = Nuclear.Model(8.)
    refConfigs  = [Configuration("[He] 2s^2 2p^4")]
    orbitals    = wb["mean-field basis"].orbitals #   get a proper set of orbitals
    ciSettings  = CiSettings(true, false, Int64[], false, LevelSymmetry[] )
    from        = [Shell("2s")]
    to          = [Shell("2s"), Shell("2p")]
    excitations = RasStep(RasStep(), seFrom=from, seTo=to, deFrom=from, deTo=to, frozen=[Shell("1s")])
    Representation(name, nuclearM, grid, refConfigs, CiExpansion(orbitals, excitations, ciSettings) )
```

`AtomicState.Representation( ... example for the computation of a restricted-active-space (RAS) expansion)`  

```
    name        = "Beryllium 1s^2 2s^2 ^1S_0 ground state"
    refConfigs  = [Configuration("[He] 2s^2")]
    rasSettings = RasSettings([1], 24, 1.0e-6, CoulombInteraction(), true, [1,2,3] )
    from        = [Shell("2s")]
    
    frozen      = [Shell("1s")]
    to          = [Shell("2s"), Shell("2p")]
    step1       = RasStep(RasStep(), seFrom=from, seTo=deepcopy(to), deFrom=from, deTo=deepcopy(to), 
                          frozen=deepcopy(frozen))

    append!(frozen, [Shell("2s"), Shell("2p")])
    append!(to,     [Shell("3s"), Shell("3p"), Shell("3d")])
    step2       = RasStep(step1; seTo=deepcopy(to), deTo=deepcopy(to), frozen=deepcopy(frozen))

    append!(frozen, [Shell("3s"), Shell("3p"), Shell("3d")])
    append!(to,     [Shell("4s"), Shell("4p"), Shell("4d"), Shell("4f")])
    step3       = RasStep(step2, seTo=deepcopy(to), deTo=deepcopy(to), frozen=deepcopy(frozen))

    Representation(name, Nuclear.Model(4.), Radial.Grid(true), refConfigs, 
                   RasExpansion(LevelSymmetry(0, Basics.plus), 4, [step1, step2, step3], rasSettings) )
```

`AtomicState.Representation( ... example for the computation of Green(function) expansion)`  

```
    name            = "Lithium 1s^2 2s ground configuration"
    refConfigs      = [Configuration("[He] 2s")]
    levelSymmetries = [LevelSymmetry(1//2, Basics.plus), LevelSymmetry(3//2, Basics.plus)]
    greenSettings   = GreenSettings(5, [0, 1, 2], 0.01, true, false, Int64[])
    Representation(name, Nuclear.Model(8.), Radial.Grid(true), refConfigs, 
                   GreenExpansion( AtomicState.DampedSpaceCI(), Basics.DeExciteSingleElectron(), 
                   levelSymmetries, 3, greenSettings) ) 
                   
... These simple examples can be further improved by overwriting the corresponding parameters.
```


For a RAS expansion, the `repType` must be a `RasExpansion()`, while all virtual excitations of the stepwise enlarged computations refer to the (list of) reference configurations `refConfigs`.

In [3]:
? AtomicState.RasExpansion

`struct  AtomicState.RasExpansion    <:  AbstractRepresentationType`       ... a struct to represent (and generate) a restricted active-space representation.

```
+ symmetries       ::Array{LevelSymmetry,1}         ... Symmetries of the levels/CSF in the many-electron basis.
+ NoElectrons      ::Int64                          ... Number of electrons.
+ steps            ::Array{AtomicState.RasStep,1}   ... List of SCF steps that are to be done in this model 
                                                        computation.
+ settings         ::AtomicState.RasSettings        ... Settings for the given RAS computation
```

---

`AtomicState.RasExpansion()`  ... constructor for an 'empty' instance of the a variable::AtomicState.RasExpansion


In this RAS expansion, we have to specify the (level) `symmetry` of automtically generated CSF expansions as well as *list of* `steps`, which specify the virtual excitations as well as those orbitals which are kept to be frozen during this particular step. These steps are *decribed* by:

In [4]:
? RasStep

search: RasStep RasSettings uppercasefirst lowercasefirst



`struct  AtomicState.RasStep`       ... specifies an individual step of a (relativistic) restricted active space computation for a set of levels. This struct          comprises all information to generate the orbital basis and to perform the associated SCF and multiplet computations for a          selected number of levels.

```
+ seFrom            ::Array{Shell,1}        ... Single-excitations from shells   [sh_1, sh_2, ...]
+ seTo              ::Array{Shell,1}        ... Single-excitations to shells  [sh_1, sh_2, ...]
+ deFrom            ::Array{Shell,1}        ... Double-excitations from shells   [sh_1, sh_2, ...]
+ deTo              ::Array{Shell,1}        ... Double-excitations to shells  [sh_1, sh_2, ...]
+ teFrom            ::Array{Shell,1}        ... Triple-excitations from shells   [sh_1, sh_2, ...]
+ teTo              ::Array{Shell,1}        ... Triple-excitations to shells  [sh_1, sh_2, ...]
+ qeFrom            ::Array{Shell,1}        ... Quadrupole-excitations from shells   [sh_1, sh_2, ...]
+ qeTo              ::Array{Shell,1}        ... Quadrupole-excitations to shells  [sh_1, sh_2, ...]
+ frozenShells      ::Array{Shell,1}        ... List of shells that are kept 'frozen' in this step.
+ constraints       ::Array{String,1}       ... List of Strings to define 'constraints/restrictions' 
                                                to the generated CSF basis.
```

---

`AtomicState.RasStep()`  ... constructor for an 'empty' instance of a variable::AtomicState.RasStep

---

`AtomicState.RasStep(rasStep::AtomicState.RasStep;`

```
                seFrom::Array{Shell,1}=Shell[], seTo::Array{Shell,1}=Shell[], 
                deFrom::Array{Shell,1}=Shell[], deTo::Array{Shell,1}=Shell[], 
                teFrom::Array{Shell,1}=Shell[], teTo::Array{Shell,1}=Shell[], 
                qeFrom::Array{Shell,1}=Shell[], qeTo::Array{Shell,1}=Shell[], 
                frozen::Array{Shell,1}=Shell[], constraints::Array{String,1}=String[]  
                
... constructor for modifying the given rasStep by specifying all excitations, frozen shells and 
    constraints optionally.
```


with quite an obvious meaning of single excitations (se), double excitations (de), etc., and where `from` and `to` refer lists of active shells. For each step, we can also specify which `frozenShells` are kept frozen and which additional constraints should be taken into account in generating the CSF lists. **We note, however, that presently no constraints are taken into account in these RAS expansions.** 

Apart from the `steps`, various details about the RAS computations can be specified in terms of the `settings::RasSettings` above, and which apply (equally) to *all* steps:

In [5]:
? RasSettings

search: RasSettings AbstractProcessSettings



`struct  AtomicState.RasSettings`       ... a struct for defining the settings for a restricted active-space computations.

```
+ levelsScf            ::Array{Int64,1}         ... Levels on which the optimization need to be carried out.
+ maxIterationsScf     ::Int64                  ... maximum number of SCF iterations in each RAS step.
+ accuracyScf          ::Float64                ... convergence criterion for the SCF field.
+ eeInteractionCI      ::AbstractEeInteraction  ... logical flag to include Breit interactions.
+ levelSelectionCI     ::LevelSelection         ... Specifies the selected levels, if any.
```

---

`AtomicState.RasSettings()`  ... constructor for setting the default values.


These settings enables one to specify the levels on which the SCF is optimized as well as the maximum number of iterations and requested accuracy in these computations. In addition, the flag `breitCI` determines the use of the Breit interaction in the set-up (and diagonalization) of the Hamiltonian matrix; this flag does not affect the SCF of the *next* RAS step but may influence further computations, if the multiplets of the individual steps are subsequently used in computations of properties and processes. We can look for the default settings as well as specify these settings explictly by:

In [6]:
AtomicState.RasSettings()

levelsScf:            [1]  
maxIterationsScf:     24  
accuracyScf:          1.0e-6  
eeInteractionCI:      CoulombInteraction()  
levelSelectionCI:     Inactive LevelSelection.  


In [7]:
rasSettings = RasSettings([1], 24, 1.0e-6, CoulombInteraction(), LevelSelection(true, indices=[1,2,3]) )

levelsScf:            [1]  
maxIterationsScf:     24  
accuracyScf:          1.0e-6  
eeInteractionCI:      CoulombInteraction()  
levelSelectionCI:     LevelSelection:  indices = [1, 2, 3];    symmetries = LevelSymmetry[].  


The last two entries to these settings tells JAC that we wish to compute (and print) always the lowest three levels of each Hamiltonian matrix; **this feature is not done properly at present since, until now, the whole Hamiltonian matrix is diagonalized (and printed) in all cases.**

We are now prepared to define the RAS expansion as a whole, i.e. a name (mainly for the intermediate and final printout), the reference configuration(s) as well as in turn all (three) steps of a RAS computation for the $1s^2 2s^2\: ^1S_0$ ground level of neutral beryllium:

In [8]:
name        = "Beryllium 1s^2 2s^2 ^1S_0 ground state"
refConfigs  = [Configuration("[He] 2s^2")]

1-element Vector{Configuration}:
 Configuration: 1s^2 2s^2 

We also specify that we will keep the $1s$ orbital frozen in the computation of all layers beyond those in the reference configurations as well as that excitations are to be taken from the $2s$ shell:

In [9]:
frozen      = [Shell("1s")]
from        = [Shell("2s")]

1-element Vector{Shell}:
 2s

In [10]:
to          = [Shell("2s"), Shell("2p")]
step1       = RasStep(RasStep(); seFrom=from, seTo=deepcopy(to), deFrom=from, deTo=deepcopy(to), frozen=deepcopy(frozen))


CI or RAS step with 1 (explicitly) frozen shell(s): Shell[1s]  ... and virtual excitations
   Singles from:          { 2s }   ... to { 2s, 2p }
   Doubles from:          { 2s }   ... to { 2s, 2p }


Similarly, we can specify the second and third step by using (and overwriting) the information provided for the previous step; the use of deepcopy() ensures that no internal calls are repeated if these cells are called in different order:

In [11]:
append!(frozen, [Shell("2s"), Shell("2p")])
append!(to,     [Shell("3s"), Shell("3p"), Shell("3d")])
step2       = RasStep(step1; seTo=deepcopy(to), deTo=deepcopy(to), frozen=deepcopy(frozen))


CI or RAS step with 3 (explicitly) frozen shell(s): Shell[1s, 2s, 2p]  ... and virtual excitations
   Singles from:          { 2s }   ... to { 2s, 2p, 3s, 3p, 3d }
   Doubles from:          { 2s }   ... to { 2s, 2p, 3s, 3p, 3d }


In [12]:
append!(frozen, [Shell("3s"), Shell("3p"), Shell("3d")])
append!(to,     [Shell("4s"), Shell("4p"), Shell("4d"), Shell("4f")])
step3       = RasStep(step2; seTo=deepcopy(to), deTo=deepcopy(to), frozen=deepcopy(frozen))


CI or RAS step with 6 (explicitly) frozen shell(s): Shell[1s, 2s, 2p, 3s, 3p, 3d]  ... and virtual excitations
   Singles from:          { 2s }   ... to { 2s, 2p, 3s, 3p, 3d, 4s, 4p, 4d, 4f }
   Doubles from:          { 2s }   ... to { 2s, 2p, 3s, 3p, 3d, 4s, 4p, 4d, 4f }


With the definition of all *steps*, we can now define the RAS expansion as a whole and subsequently *generate* it:

In [13]:
wa          = Representation(name, Nuclear.Model(4.), Radial.Grid(true), refConfigs, 
                             RasExpansion([LevelSymmetry(0, Basics.plus)], 4, [step1, step2, step3], rasSettings) )

Atomic representation:   Beryllium 1s^2 2s^2 ^1S_0 ground state for Z = 4.0 and with reference configurations: 
   1s^2 2s^2 ,  
representation type:   RAS expansion for symmétry LevelSymmetry[0 +] and with 3 steps:
RasStep[
CI or RAS step with 1 (explicitly) frozen shell(s): Shell[1s]  ... and virtual excitations
   Singles from:          { 2s }   ... to { 2s, 2p }
   Doubles from:          { 2s }   ... to { 2s, 2p }
, 
CI or RAS step with 3 (explicitly) frozen shell(s): Shell[1s, 2s, 2p]  ... and virtual excitations
   Singles from:          { 2s }   ... to { 2s, 2p, 3s, 3p, 3d }
   Doubles from:          { 2s }   ... to { 2s, 2p, 3s, 3p, 3d }
, 
CI or RAS step with 6 (explicitly) frozen shell(s): Shell[1s, 2s, 2p, 3s, 3p, 3d]  ... and virtual excitations
   Singles from:          { 2s }   ... to { 2s, 2p, 3s, 3p, 3d, 4s, 4p, 4d, 4f }
   Doubles from:          { 2s }   ... to { 2s, 2p, 3s, 3p, 3d, 4s, 4p, 4d, 4f }
]  
... and the current settings:
levelsScf:            [1]  
maxItera

In [14]:
wb = generate(wa, output=true)


... in performSCF ...
>> include Configuration: 1s_1/2^2 2s_1/2^2 
(Re-) Define a new standard subshell list.
Start SCF process with hydrogenic orbitals.
>> (Re-) Define a storage array for various B-spline matrices:
Nuclear model = Fermi nuclear model for Z = 4.0 with mass = 8.08, radius R = 2.247554858082373 fm and nuclear spin I = 0, dipole moment mu = 0.0 and quadrupole moment Q = 0.0. 
Generate hydrogenic orbital for subshell 1s_1/2 
  -----------------------------------------------------------------------------
   Index    Subshell     Energies [a.u.]    Dirac-E  [a.u.]     Delta-E / |E|    
  -----------------------------------------------------------------------------
      1      1s_1/2      -8.00170451e+00    -8.00170477e+00    +3.20928495e-08    
      2      2s_1/2      -2.00053315e+00    -2.00053275e+00    -1.98702657e-07    
      3      3s_1/2      -8.89088301e-01    -8.89078304e-01    -1.12449326e-05    
      4      4s_1/2      -5.00125339e-01    -5.00086566e-01    -7

  1s_1/2::  en [a.u.] = -3.7931595e+00;   self-cons'cy = 7.9708e-08  [3.0112e-06 for sym-block kappa = -1]
  2s_1/2::  en [a.u.] = -1.6992654e-01;   self-cons'cy = 5.1916e-07  [3.0112e-06 for sym-block kappa = -1]

Iteration 18 for symmetries ... 
  1s_1/2::  en [a.u.] = -3.7931597e+00;   self-cons'cy = 2.9969e-08  [1.0901e-06 for sym-block kappa = -1]
  2s_1/2::  en [a.u.] = -1.6992661e-01;   self-cons'cy = 1.9519e-07  [1.0901e-06 for sym-block kappa = -1]

Iteration 19 for symmetries ... 
  1s_1/2::  en [a.u.] = -3.7931596e+00;   self-cons'cy = 1.1267e-08  [4.7654e-07 for sym-block kappa = -1]
  2s_1/2::  en [a.u.] = -1.6992658e-01;   self-cons'cy = 7.3413e-08  [4.7654e-07 for sym-block kappa = -1]
Compute CI matrix of dimension 1 x 1 for the symmetry 0^+ ...   ... done.

  Eigenenergies:

  Level  J Parity          Hartrees                    eV                   [eV]

     1    0 +     -1.457046946562298e+01    -3.964826691216900e+02    -3.964826691216900e+02 

  Energy of each lev

  3s_1/2::  en [a.u.] = -1.2449498e-03;   self-cons'cy = 9.9720e-01  [1.0000e+02 for sym-block kappa = -1]
  3p_1/2::  en [a.u.] = -9.4453944e-03;   self-cons'cy = 9.7898e-01  [1.0000e+02 for sym-block kappa = 1]
  3p_3/2::  en [a.u.] = -2.1778579e-02;   self-cons'cy = 9.5217e-01  [1.0000e+02 for sym-block kappa = -2]
  3d_3/2::  en [a.u.] = -1.7151397e-02;   self-cons'cy = 9.6214e-01  [1.0000e+02 for sym-block kappa = 2]
  3d_5/2::  en [a.u.] = -2.5757737e-02;   self-cons'cy = 9.4368e-01  [1.0000e+02 for sym-block kappa = -3]

Iteration 2 for symmetries ... 
  3s_1/2::  en [a.u.] = -8.2619489e-02;   self-cons'cy = 9.7031e-01  [2.8133e+00 for sym-block kappa = -1]
  3p_1/2::  en [a.u.] = -4.4132095e-02;   self-cons'cy = 6.4741e-01  [6.4741e-01 for sym-block kappa = 1]
  3p_3/2::  en [a.u.] = -3.5572352e-02;   self-cons'cy = 2.4052e-01  [1.1443e+00 for sym-block kappa = -2]
  3d_3/2::  en [a.u.] = -1.1462046e-02;   self-cons'cy = 1.9883e-01  [1.0344e+00 for sym-block kappa = 2]
  3d_5/2


Iteration 1 for symmetries ... 
  4s_1/2::  en [a.u.] = -3.5705789e-03;   self-cons'cy = 9.8582e-01  [1.0000e+02 for sym-block kappa = -1]
  4p_1/2::  en [a.u.] = -4.8816291e-03;   self-cons'cy = 9.8067e-01  [1.0000e+02 for sym-block kappa = 1]
  4p_3/2::  en [a.u.] = -8.9713420e-03;   self-cons'cy = 9.6475e-01  [1.0000e+02 for sym-block kappa = -2]
  4d_3/2::  en [a.u.] = -7.1007230e-03;   self-cons'cy = 9.7200e-01  [1.0000e+02 for sym-block kappa = 2]
  4d_5/2::  en [a.u.] = -9.6690557e-03;   self-cons'cy = 9.6206e-01  [1.0000e+02 for sym-block kappa = -3]
  4f_5/2::  en [a.u.] = -5.9514792e-03;   self-cons'cy = 9.7647e-01  [1.0000e+02 for sym-block kappa = 3]
  4f_7/2::  en [a.u.] = -7.6431081e-03;   self-cons'cy = 9.6989e-01  [1.0000e+02 for sym-block kappa = -4]

Iteration 2 for symmetries ... 
  4s_1/2::  en [a.u.] = -3.1676886e-02;   self-cons'cy = 7.9740e-01  [1.1108e+00 for sym-block kappa = -1]
  4p_1/2::  en [a.u.] = -2.1405825e-02;   self-cons'cy = 6.2860e-01  [9.9456e-01 

  4p_1/2::  en [a.u.] = -1.7672079e-02;   self-cons'cy = 7.5676e-10  [3.6254e-07 for sym-block kappa = 1]
  4p_3/2::  en [a.u.] = -1.7669161e-02;   self-cons'cy = 1.8558e-10  [3.1189e-07 for sym-block kappa = -2]
  4d_3/2::  en [a.u.] = -1.0868792e-02;   self-cons'cy = 2.9283e-10  [3.6556e-07 for sym-block kappa = 2]
  4d_5/2::  en [a.u.] = -1.0868539e-02;   self-cons'cy = 6.3547e-10  [2.6479e-07 for sym-block kappa = -3]
  4f_5/2::  en [a.u.] = -4.1141892e-03;   self-cons'cy = 5.3166e-10  [2.1482e-07 for sym-block kappa = 3]
  4f_7/2::  en [a.u.] = -4.1141779e-03;   self-cons'cy = 2.0712e-09  [4.8007e-08 for sym-block kappa = -4]

Iteration 13 for symmetries ... 
  4s_1/2::  en [a.u.] = -2.6195374e-02;   self-cons'cy = 1.9067e-10  [1.9140e-06 for sym-block kappa = -1]
  4p_1/2::  en [a.u.] = -1.7672079e-02;   self-cons'cy = 1.7688e-10  [5.4555e-07 for sym-block kappa = 1]
  4p_3/2::  en [a.u.] = -1.7669161e-02;   self-cons'cy = 1.7590e-10  [1.2838e-07 for sym-block kappa = -2]
  4d_3/

Dict{String, Any} with 4 entries:
  "reference multiplet" => name:        Reference multiplet:  …
  "step2"               => name:        Multiplet:  …
  "step3"               => name:        Multiplet:  …
  "step1"               => name:        Multiplet:  …

Of course, analogue computations can be readily carried out also for the (two) $1s^2 2s2p\: ^{1,3}P_1$ excited levels of neutral beryllium, and where we use major parts of the previous computation:

In [15]:
name2        = "Beryllium 1s^2 2s 2p ^1,3P_1 excited levels"
refConfigs2  = [Configuration("[He] 2s 2p")]
wa2          = Representation(name2, Nuclear.Model(4.), Radial.Grid(true), refConfigs2, 
                              RasExpansion([LevelSymmetry(1, Basics.minus)], 4, [step1, step2, step3], rasSettings) )

Atomic representation:   Beryllium 1s^2 2s 2p ^1,3P_1 excited levels for Z = 4.0 and with reference configurations: 
   1s^2 2s^1 2p^1 ,  
representation type:   RAS expansion for symmétry LevelSymmetry[1 -] and with 3 steps:
RasStep[
CI or RAS step with 1 (explicitly) frozen shell(s): Shell[1s]  ... and virtual excitations
   Singles from:          { 2s }   ... to { 2s, 2p }
   Doubles from:          { 2s }   ... to { 2s, 2p }
, 
CI or RAS step with 3 (explicitly) frozen shell(s): Shell[1s, 2s, 2p]  ... and virtual excitations
   Singles from:          { 2s }   ... to { 2s, 2p, 3s, 3p, 3d }
   Doubles from:          { 2s }   ... to { 2s, 2p, 3s, 3p, 3d }
, 
CI or RAS step with 6 (explicitly) frozen shell(s): Shell[1s, 2s, 2p, 3s, 3p, 3d]  ... and virtual excitations
   Singles from:          { 2s }   ... to { 2s, 2p, 3s, 3p, 3d, 4s, 4p, 4d, 4f }
   Doubles from:          { 2s }   ... to { 2s, 2p, 3s, 3p, 3d, 4s, 4p, 4d, 4f }
]  
... and the current settings:
levelsScf:            [1] 

In [16]:
wb2 = generate(wa2, output=true)


... in performSCF ...
>> include Configuration: 1s_1/2^2 2s_1/2^1 2p_1/2^0 2p_3/2^1 
>> include Configuration: 1s_1/2^2 2s_1/2^1 2p_1/2^1 2p_3/2^0 
(Re-) Define a new standard subshell list.
Start SCF process with hydrogenic orbitals.
>> (Re-) Define a storage array for various B-spline matrices:
Nuclear model = Fermi nuclear model for Z = 4.0 with mass = 8.08, radius R = 2.247554858082373 fm and nuclear spin I = 0, dipole moment mu = 0.0 and quadrupole moment Q = 0.0. 
Generate hydrogenic orbital for subshell 1s_1/2 
  -----------------------------------------------------------------------------
   Index    Subshell     Energies [a.u.]    Dirac-E  [a.u.]     Delta-E / |E|    
  -----------------------------------------------------------------------------
      1      1s_1/2      -8.00170451e+00    -8.00170477e+00    +3.20928495e-08    
      2      2s_1/2      -2.00053315e+00    -2.00053275e+00    -1.98702657e-07    
      3      3s_1/2      -8.89088301e-01    -8.89078304e-01    -1.1

     21     22p_3/2      +1.52166308e+00    -1.65292239e-02    +1.01086260e+00    
     22     23p_3/2      +5.03865484e+00    -1.51231352e-02    +1.00300142e+00    
     23     24p_3/2      +1.31111521e+01    -1.38891200e-02    +1.00105934e+00    
     24     25p_3/2      +3.06939192e+01    -1.28002050e-02    +1.00041703e+00    
     25     26p_3/2      +6.79042970e+01    -1.18345023e-02    +1.00017428e+00    
     26     27p_3/2      +1.45204025e+02    -1.09741004e-02    +1.00007558e+00    
     27     28p_3/2      +3.03523170e+02    -1.02042286e-02    +1.00003362e+00    
     28     29p_3/2      +6.23374451e+02    -9.51261766e-03    +1.00001526e+00    
     29     30p_3/2      +1.25861393e+03    -8.88900881e-03    +1.00000706e+00    
     30     31p_3/2      +2.48881647e+03    -8.32477068e-03    +1.00000334e+00    
     31     32p_3/2      +4.78239247e+03    -7.81259914e-03    +1.00000163e+00    
     32     33p_3/2      +8.84163228e+03    -7.34627969e-03    +1.00000083e+00    
    

  2p_3/2::  en [a.u.] = -6.5903255e-02;   self-cons'cy = 1.3677e-09  [3.2540e-07 for sym-block kappa = -2]
>>> Sign changed for orbital 2p_1/2
>>> Sign changed for orbital 2p_3/2
Compute CI matrix of dimension 1 x 1 for the symmetry 2^- ...   ... done.
Compute CI matrix of dimension 1 x 1 for the symmetry 0^- ...   ... done.
Compute CI matrix of dimension 2 x 2 for the symmetry 1^- ...   ... done.

  Eigenenergies:

  Level  J Parity          Hartrees                    eV                   [eV]

     1    0 -     -1.450426218730207e+01    -3.946810773139661e+02    -3.946810773139661e+02 
     2    1 -     -1.450425255076645e+01    -3.946808150904754e+02    -3.946808150904754e+02 
     3    2 -     -1.450423327696930e+01    -3.946802906237411e+02    -3.946802906237411e+02 
     4    1 -     -1.437763916831300e+01    -3.912354894652368e+02    -3.912354894652368e+02 

  Energy of each level relative to immediately lower level:

  Level  J Parity          Hartrees                    eV   

  3p_1/2::  en [a.u.] = -2.3441558e-02;   self-cons'cy = 2.8182e-03  [2.1819e-02 for sym-block kappa = 1]
  3p_3/2::  en [a.u.] = -2.3436221e-02;   self-cons'cy = 2.8185e-03  [2.1825e-02 for sym-block kappa = -2]
  3d_3/2::  en [a.u.] = -7.1920837e-03;   self-cons'cy = 8.0796e-03  [1.3622e-02 for sym-block kappa = 2]
  3d_5/2::  en [a.u.] = -7.2072962e-03;   self-cons'cy = 5.0201e-03  [5.0201e-03 for sym-block kappa = -3]

Iteration 6 for symmetries ... 
  3s_1/2::  en [a.u.] = -4.5014648e-02;   self-cons'cy = 2.8169e-04  [3.7596e-03 for sym-block kappa = -1]
  3p_1/2::  en [a.u.] = -2.3449617e-02;   self-cons'cy = 1.7187e-04  [8.1852e-04 for sym-block kappa = 1]
  3p_3/2::  en [a.u.] = -2.3444279e-02;   self-cons'cy = 1.7189e-04  [8.1771e-04 for sym-block kappa = -2]
  3d_3/2::  en [a.u.] = -7.1992799e-03;   self-cons'cy = 5.0004e-04  [5.0004e-04 for sym-block kappa = 2]
  3d_5/2::  en [a.u.] = -7.1973885e-03;   self-cons'cy = 6.8782e-04  [9.2970e-04 for sym-block kappa = -3]

Iterati

  3d_5/2::  en [a.u.] = -7.1982547e-03;   self-cons'cy = 1.1493e-08  [1.8776e-05 for sym-block kappa = -3]

Iteration 20 for symmetries ... 
  3s_1/2::  en [a.u.] = -4.5012240e-02;   self-cons'cy = 1.3503e-08  [9.0829e-05 for sym-block kappa = -1]
  3p_1/2::  en [a.u.] = -2.3448645e-02;   self-cons'cy = 2.0048e-08  [6.2921e-05 for sym-block kappa = 1]
  3p_3/2::  en [a.u.] = -2.3443307e-02;   self-cons'cy = 1.9789e-08  [6.3108e-05 for sym-block kappa = -2]
  3d_3/2::  en [a.u.] = -7.1984215e-03;   self-cons'cy = 5.7070e-08  [2.7762e-05 for sym-block kappa = 2]
  3d_5/2::  en [a.u.] = -7.1982554e-03;   self-cons'cy = 5.3168e-08  [1.9363e-05 for sym-block kappa = -3]

Iteration 21 for symmetries ... 
  3s_1/2::  en [a.u.] = -4.5012239e-02;   self-cons'cy = 1.1845e-08  [6.2096e-05 for sym-block kappa = -1]
  3p_1/2::  en [a.u.] = -2.3448644e-02;   self-cons'cy = 1.6243e-08  [4.2940e-05 for sym-block kappa = 1]
  3p_3/2::  en [a.u.] = -2.3443307e-02;   self-cons'cy = 1.6019e-08  [4.3508e-0

  4s_1/2::  en [a.u.] = -1.6102958e-02;   self-cons'cy = 3.4943e-03  [5.8796e-01 for sym-block kappa = -1]
  4p_1/2::  en [a.u.] = -1.0320162e-02;   self-cons'cy = 4.5163e-03  [7.3402e-02 for sym-block kappa = 1]
  4p_3/2::  en [a.u.] = -1.0319011e-02;   self-cons'cy = 4.5164e-03  [7.3225e-02 for sym-block kappa = -2]
  4d_3/2::  en [a.u.] = -5.3715074e-03;   self-cons'cy = 9.7442e-03  [2.1972e+01 for sym-block kappa = 2]
  4d_5/2::  en [a.u.] = -5.3587741e-03;   self-cons'cy = 1.4195e-03  [7.9077e-02 for sym-block kappa = -3]
  4f_5/2::  en [a.u.] = -2.0252725e-03;   self-cons'cy = 2.1197e-03  [5.3604e-03 for sym-block kappa = 3]
  4f_7/2::  en [a.u.] = -2.0253986e-03;   self-cons'cy = 2.1098e-03  [5.3601e-03 for sym-block kappa = -4]

Iteration 5 for symmetries ... 
  4s_1/2::  en [a.u.] = -1.6094394e-02;   self-cons'cy = 2.6599e-04  [7.3304e-02 for sym-block kappa = -1]
  4p_1/2::  en [a.u.] = -1.0308896e-02;   self-cons'cy = 5.4610e-04  [1.8596e-02 for sym-block kappa = 1]
  4p_3/2

Dict{String, Any} with 4 entries:
  "reference multiplet" => name:        Reference multiplet:  …
  "step2"               => name:        Multiplet:  …
  "step3"               => name:        Multiplet:  …
  "step1"               => name:        Multiplet:  …

All *results* of this and the previous RAS expansion are kept by the (two) dictionaries `wb2` and `wb` above. We can use these variables to extract, for instance, the multiplets as obtained from the first step:

In [17]:
wm = wb["reference multiplet"];   wm2 = wb2["reference multiplet"]

name:        Reference multiplet:  
levels:      Level[Level: J = 0, M = 0, parity = minus, index = 1 
energy:         -14.504262187302068  
relativeOcc:    0.0  
hasStateRep:    true  
basis:           (level.basis)  
mc:             [0.0, 0.0, 1.0, 0.0]  
, Level: J = 1, M = 1, parity = minus, index = 2 
energy:         -14.504252550766449  
relativeOcc:    0.0  
hasStateRep:    true  
basis:           (level.basis)  
mc:             [-0.5772910115679837, 0.0, 0.0, 0.8165384791685045]  
, Level: J = 2, M = 2, parity = minus, index = 3 
energy:         -14.504233276969297  
relativeOcc:    0.0  
hasStateRep:    true  
basis:           (level.basis)  
mc:             [0.0, 1.0, 0.0, 0.0]  
, Level: J = 1, M = 1, parity = minus, index = 4 
energy:         -14.377639168313001  
relativeOcc:    0.0  
hasStateRep:    true  
basis:           (level.basis)  
mc:             [-0.8165384791685045, 0.0, 0.0, -0.5772910115679837]  
]  


Obviously, these (reference) multiplets describe (three) different *level* since they are built from the $1s^2 2s^2$ and $1s^2 2s 2p$ configurations:

In [18]:
println("Energies from references conf:  $(wm.levels[1].energy)   $(wm2.levels[1].energy)   $(wm2.levels[2].energy)   ")

Energies from references conf:  -14.57046946562298   -14.504262187302068   -14.504252550766449   


We can see how these energies are *lowered* or, at least, modified due to the incorporation of additional correlation layers with each step:

In [19]:
wm = wb["step1"];   wm2 = wb2["step1"]
println("Energies from step 1:           $(wm.levels[1].energy)   $(wm2.levels[1].energy)   $(wm2.levels[2].energy)   ")

Energies from step 1:           -14.605107880800258   -14.50425136940429   -14.37764020608959   


In [20]:
wm = wb["step2"];   wm2 = wb2["step2"]
println("Energies from step 2:           $(wm.levels[1].energy)   $(wm2.levels[1].energy)   $(wm2.levels[2].energy)   ")

Energies from step 2:           -14.615971463534336   -14.50528076682614   -14.378196809740437   


In [21]:
wm = wb["step3"];   wm2 = wb2["step3"]
println("Energies from step 3:           $(wm.levels[1].energy)   $(wm2.levels[1].energy)   $(wm2.levels[2].energy)  ")

Energies from step 3:           -14.616790203099159   -14.505769505124787   -14.378384474908552  


Finally, we could test how much we benefit by starting from the $1s^2 (2s^2 + 2p^2)$ multi-reference configuration but, otherwise, by performing analogue steps. In this case, of course, we need to allow excitations from $2p$ and can skip step$_2$ since these excitations are now included already into the reference configurations:

In [22]:
frozen       = [Shell("1s"), Shell("2s"), Shell("2p")]
from         = [Shell("2s"), Shell("2p")]

2-element Vector{Shell}:
 2s
 2p

In [23]:
to          = [Shell("2s"), Shell("2p"), Shell("3s"), Shell("3p"), Shell("3d")]
step2       = RasStep(RasStep(); seFrom=from, seTo=deepcopy(to), deFrom=from, deTo=deepcopy(to), frozen=deepcopy(frozen))


CI or RAS step with 3 (explicitly) frozen shell(s): Shell[1s, 2s, 2p]  ... and virtual excitations
   Singles from:          { 2s, 2p }   ... to { 2s, 2p, 3s, 3p, 3d }
   Doubles from:          { 2s, 2p }   ... to { 2s, 2p, 3s, 3p, 3d }


In [24]:
append!(frozen, [Shell("3s"), Shell("3p"), Shell("3d")])
append!(to,     [Shell("4s"), Shell("4p"), Shell("4d"), Shell("4f")])
step3       = RasStep(step2; seTo=deepcopy(to), deTo=deepcopy(to), frozen=deepcopy(frozen))


CI or RAS step with 6 (explicitly) frozen shell(s): Shell[1s, 2s, 2p, 3s, 3p, 3d]  ... and virtual excitations
   Singles from:          { 2s, 2p }   ... to { 2s, 2p, 3s, 3p, 3d, 4s, 4p, 4d, 4f }
   Doubles from:          { 2s, 2p }   ... to { 2s, 2p, 3s, 3p, 3d, 4s, 4p, 4d, 4f }


In [25]:
name3        = "Beryllium 1s^2 (2s^2 + 2p^2) ^1S_0 ground state with multi-reference configurations"
refConfigs3  = [Configuration("[He] 2s^2"), Configuration("[He] 2p^2")]
wa3          = Representation(name3, Nuclear.Model(4.), Radial.Grid(true), refConfigs3, 
                              RasExpansion([LevelSymmetry(0, Basics.plus)], 4, [step2, step3], rasSettings) )

Atomic representation:   Beryllium 1s^2 (2s^2 + 2p^2) ^1S_0 ground state with multi-reference configurations for Z = 4.0 and with reference configurations: 
   1s^2 2s^2 ,  1s^2 2p^2 ,  
representation type:   RAS expansion for symmétry LevelSymmetry[0 +] and with 2 steps:
RasStep[
CI or RAS step with 3 (explicitly) frozen shell(s): Shell[1s, 2s, 2p]  ... and virtual excitations
   Singles from:          { 2s, 2p }   ... to { 2s, 2p, 3s, 3p, 3d }
   Doubles from:          { 2s, 2p }   ... to { 2s, 2p, 3s, 3p, 3d }
, 
CI or RAS step with 6 (explicitly) frozen shell(s): Shell[1s, 2s, 2p, 3s, 3p, 3d]  ... and virtual excitations
   Singles from:          { 2s, 2p }   ... to { 2s, 2p, 3s, 3p, 3d, 4s, 4p, 4d, 4f }
   Doubles from:          { 2s, 2p }   ... to { 2s, 2p, 3s, 3p, 3d, 4s, 4p, 4d, 4f }
]  
... and the current settings:
levelsScf:            [1]  
maxIterationsScf:     24  
accuracyScf:          1.0e-6  
eeInteractionCI:      CoulombInteraction()  
levelSelectionCI:     LevelSele

In [26]:
wb3 = generate(wa3, output=true)


... in performSCF ...
>> include Configuration: 1s_1/2^2 2s_1/2^2 
>> include Configuration: 1s_1/2^2 2p_1/2^0 2p_3/2^2 
>> include Configuration: 1s_1/2^2 2p_1/2^1 2p_3/2^1 
>> include Configuration: 1s_1/2^2 2p_1/2^2 2p_3/2^0 
(Re-) Define a new standard subshell list.
Start SCF process with hydrogenic orbitals.
>> (Re-) Define a storage array for various B-spline matrices:
Nuclear model = Fermi nuclear model for Z = 4.0 with mass = 8.08, radius R = 2.247554858082373 fm and nuclear spin I = 0, dipole moment mu = 0.0 and quadrupole moment Q = 0.0. 
Generate hydrogenic orbital for subshell 1s_1/2 
  -----------------------------------------------------------------------------
   Index    Subshell     Energies [a.u.]    Dirac-E  [a.u.]     Delta-E / |E|    
  -----------------------------------------------------------------------------
      1      1s_1/2      -8.00170451e+00    -8.00170477e+00    +3.20928495e-08    
      2      2s_1/2      -2.00053315e+00    -2.00053275e+00    -1.987

     18     19p_3/2      -7.25147250e-03    -2.21611225e-02    +2.05608585e+00    
     19     20p_3/2      -6.41873759e-03    -2.00003941e-02    +2.11593889e+00    
     20     21p_3/2      +1.60564248e-01    -1.81409313e-02    +1.11298238e+00    
     21     22p_3/2      +1.52166308e+00    -1.65292239e-02    +1.01086260e+00    
     22     23p_3/2      +5.03865484e+00    -1.51231352e-02    +1.00300142e+00    
     23     24p_3/2      +1.31111521e+01    -1.38891200e-02    +1.00105934e+00    
     24     25p_3/2      +3.06939192e+01    -1.28002050e-02    +1.00041703e+00    
     25     26p_3/2      +6.79042970e+01    -1.18345023e-02    +1.00017428e+00    
     26     27p_3/2      +1.45204025e+02    -1.09741004e-02    +1.00007558e+00    
     27     28p_3/2      +3.03523170e+02    -1.02042286e-02    +1.00003362e+00    
     28     29p_3/2      +6.23374451e+02    -9.51261766e-03    +1.00001526e+00    
     29     30p_3/2      +1.25861393e+03    -8.88900881e-03    +1.00000706e+00    
    

  2p_3/2::  en [a.u.] = -8.0110857e-02;   self-cons'cy = 1.3028e-06  [1.3028e-06 for sym-block kappa = -2]

Iteration 16 for symmetries ... 
  1s_1/2::  en [a.u.] = -3.8882444e+00;   self-cons'cy = 4.3125e-08  [2.6151e-06 for sym-block kappa = -1]
  2s_1/2::  en [a.u.] = -2.0898353e-01;   self-cons'cy = 3.4909e-07  [2.6151e-06 for sym-block kappa = -1]
  2p_1/2::  en [a.u.] = -8.0145141e-02;   self-cons'cy = 8.0251e-07  [8.0251e-07 for sym-block kappa = 1]
  2p_3/2::  en [a.u.] = -8.0110944e-02;   self-cons'cy = 5.3925e-07  [5.3925e-07 for sym-block kappa = -2]

Iteration 17 for symmetries ... 
  1s_1/2::  en [a.u.] = -3.8882441e+00;   self-cons'cy = 2.9084e-08  [1.5944e-06 for sym-block kappa = -1]
  2s_1/2::  en [a.u.] = -2.0898344e-01;   self-cons'cy = 2.1448e-07  [1.5944e-06 for sym-block kappa = -1]
  2p_1/2::  en [a.u.] = -8.0145064e-02;   self-cons'cy = 4.8325e-07  [4.8325e-07 for sym-block kappa = 1]
  2p_3/2::  en [a.u.] = -8.0110914e-02;   self-cons'cy = 1.8409e-07  [1.8409e-

  3p_3/2::  en [a.u.] = -3.2092101e-02;   self-cons'cy = 5.8462e-06  [6.4260e-05 for sym-block kappa = -2]
  3d_3/2::  en [a.u.] = -1.2022227e-02;   self-cons'cy = 1.8486e-06  [1.8486e-06 for sym-block kappa = 2]
  3d_5/2::  en [a.u.] = -1.2021693e-02;   self-cons'cy = 9.7645e-06  [9.7645e-06 for sym-block kappa = -3]

Iteration 8 for symmetries ... 
  3s_1/2::  en [a.u.] = -5.7168938e-02;   self-cons'cy = 1.0011e-06  [3.3048e-06 for sym-block kappa = -1]
  3p_1/2::  en [a.u.] = -3.2100049e-02;   self-cons'cy = 7.0605e-07  [3.6676e-06 for sym-block kappa = 1]
  3p_3/2::  en [a.u.] = -3.2092113e-02;   self-cons'cy = 1.8513e-07  [8.2601e-06 for sym-block kappa = -2]
  3d_3/2::  en [a.u.] = -1.2022256e-02;   self-cons'cy = 1.2168e-06  [1.2168e-06 for sym-block kappa = 2]
  3d_5/2::  en [a.u.] = -1.2021715e-02;   self-cons'cy = 9.4776e-07  [9.4776e-07 for sym-block kappa = -3]

Iteration 9 for symmetries ... 
  3s_1/2::  en [a.u.] = -5.7168939e-02;   self-cons'cy = 2.5122e-09  [8.8677e-07 

  4p_3/2::  en [a.u.] = -1.7654295e-02;   self-cons'cy = 1.8702e-04  [1.7730e-02 for sym-block kappa = -2]
  4d_3/2::  en [a.u.] = -1.0873210e-02;   self-cons'cy = 1.0109e-03  [1.4504e-02 for sym-block kappa = 2]
  4d_5/2::  en [a.u.] = -1.0872143e-02;   self-cons'cy = 1.1306e-03  [1.3159e-02 for sym-block kappa = -3]
  4f_5/2::  en [a.u.] = -4.1512767e-03;   self-cons'cy = 2.6965e-03  [2.6965e-03 for sym-block kappa = 3]
  4f_7/2::  en [a.u.] = -4.1505667e-03;   self-cons'cy = 1.6376e-03  [1.8467e-03 for sym-block kappa = -4]

Iteration 5 for symmetries ... 
  4s_1/2::  en [a.u.] = -2.6184891e-02;   self-cons'cy = 2.2842e-05  [1.0067e-01 for sym-block kappa = -1]
  4p_1/2::  en [a.u.] = -1.7654792e-02;   self-cons'cy = 6.9579e-05  [6.7718e-03 for sym-block kappa = 1]
  4p_3/2::  en [a.u.] = -1.7651764e-02;   self-cons'cy = 7.1692e-05  [4.8241e-03 for sym-block kappa = -2]
  4d_3/2::  en [a.u.] = -1.0870564e-02;   self-cons'cy = 1.2168e-04  [1.9694e-03 for sym-block kappa = 2]
  4d_5/2

  4f_7/2::  en [a.u.] = -4.1496294e-03;   self-cons'cy = 8.0117e-10  [1.0786e-07 for sym-block kappa = -4]

Iteration 16 for symmetries ... 
  4s_1/2::  en [a.u.] = -2.6184540e-02;   self-cons'cy = 2.3351e-10  [2.5564e-06 for sym-block kappa = -1]
  4p_1/2::  en [a.u.] = -1.7654533e-02;   self-cons'cy = 3.2142e-10  [2.6450e-07 for sym-block kappa = 1]
  4p_3/2::  en [a.u.] = -1.7651599e-02;   self-cons'cy = 2.6018e-10  [4.9063e-08 for sym-block kappa = -2]
  4d_3/2::  en [a.u.] = -1.0870385e-02;   self-cons'cy = 2.0476e-10  [2.4806e-07 for sym-block kappa = 2]
  4d_5/2::  en [a.u.] = -1.0870128e-02;   self-cons'cy = 4.4343e-10  [2.5107e-07 for sym-block kappa = -3]
  4f_5/2::  en [a.u.] = -4.1496396e-03;   self-cons'cy = 1.1678e-10  [6.1158e-08 for sym-block kappa = 3]
  4f_7/2::  en [a.u.] = -4.1496294e-03;   self-cons'cy = 1.4280e-09  [1.9594e-07 for sym-block kappa = -4]

Iteration 17 for symmetries ... 
  4s_1/2::  en [a.u.] = -2.6184540e-02;   self-cons'cy = 1.2331e-10  [1.5413e-0

Dict{String, Any} with 3 entries:
  "reference multiplet" => name:        Reference multiplet:  …
  "step2"               => name:        Multiplet:  …
  "step1"               => name:        Multiplet:  …

Again, we can print the final energy and compare the results with the single-reference configurations:

In [27]:
wm = wb["step3"];   wm3 = wb3["step2"]
println("Energies from step 3:           $(wm.levels[1].energy)   $(wm2.levels[1].energy)  ")

Energies from step 3:           -14.616790203099159   -14.505769505124787  
